# Laboratorium 1 - analiza koszykowa

## Przygotowanie

 * pobierz i wypakuj dataset: https://kaggle.com/datasets/rashikrahmanpritom/groceries-dataset-for-market-basket-analysismba?resource=download&select=basket.csv
   * alternatywnie, pobierz plik `basket.csv` z Teamsów
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab1`
 * zainstaluj potrzebne biblioteki:
 `pip install more-itertools`

## Część 1. - przygotowanie danych

In [28]:
# importujemy wszystkie potrzebne pakiety

from more_itertools import powerset

In [29]:
# definiujemy stale

PATH = './basket.csv'
EPSILON = 0.0001

In [30]:
# wczytujemy dane o koszykach

def read_baskets(path: str) -> list[tuple[str]]:
    with open(path) as f:
        raw = f.read()
    baskets = [set([y.lower() for y in x.split(',') if y]) for x in raw.split('\n')[1:] if x]
    return baskets

def unique_products(baskets: list[tuple[str]]) -> list[str]:
    products = set()
    for basket in baskets:
        products.update(basket)
    return sorted(list(products))

baskets = read_baskets(PATH)
products = unique_products(baskets)
baskets

[{'pastry', 'salty snack', 'whole milk'},
 {'sausage', 'semi-finished bread', 'whole milk', 'yogurt'},
 {'pickled vegetables', 'soda'},
 {'canned beer', 'misc. beverages'},
 {'hygiene articles', 'sausage'},
 {'rolls/buns', 'sausage', 'whole milk'},
 {'soda', 'whole milk'},
 {'frankfurter', 'soda', 'whipped/sour cream'},
 {'curd', 'frankfurter'},
 {'beef', 'white bread'},
 {'butter', 'whole milk'},
 {'frozen vegetables', 'other vegetables'},
 {'sugar', 'tropical fruit'},
 {'butter milk', 'specialty chocolate'},
 {'dental care', 'frozen meals'},
 {'rolls/buns'},
 {'detergent', 'root vegetables'},
 {'rolls/buns', 'sausage'},
 {'cling film/bags', 'dish cleaner'},
 {'canned beer', 'frozen fish'},
 {'pip fruit', 'tropical fruit', 'whole milk'},
 {'pastry', 'root vegetables', 'whole milk'},
 {'chocolate', 'red/blush wine', 'rolls/buns'},
 {'other vegetables', 'shopping bags'},
 {'chocolate', 'packaged fruit/vegetables', 'rolls/buns', 'whole milk'},
 {'hygiene articles', 'other vegetables'},
 

|## Część 2. - obliczanie wskaźników

In [23]:
# obliczamy strukture danych (np. slownik albo graf) przechowujaca wszystkie interesujace wartosci `support`

def stringify_tuple(tpl: tuple[str]):
    return ','.join(sorted(list(tpl)))

def get_supports(baskets: list[set[str]], epsilon: float, depth: int = 2):
    # calculate all supports and omit those with support < epsilon
    # support with 1 product > epsilon, then go with it to 2 products, etc.
    # if we get 2 products > epsilon, then go with it to 3 products, etc.
    # use dictionary as a data structure, key = tuple of products, value = support
    # remake the keys, so it is a string of products separated by comma - note two same sets has to has the same string
    
    result = {}
    def iterative_all_support():
        all_sets_iter = powerset(products)
        for current_tuple in all_sets_iter:
            if len(current_tuple) == 0:
                continue
            if len(current_tuple) == depth+1:
                break
    
            current_tuple_key = stringify_tuple(current_tuple)
            occurrences = 0
            for basket in baskets:
                if set(current_tuple).issubset(basket):
                    occurrences += 1
            calculated_support = occurrences / len(baskets)
            if calculated_support < epsilon:
                continue
            # print(current_tuple, calculated_support)
            if calculated_support >= epsilon:
                result[current_tuple_key] = calculated_support
    
    basket_memo = {}
    def recursive_all_support(current_products: tuple[str]):
        if len(current_products) == depth:
            return
        for product in products:
            if product in current_products: # skip already present product
                continue

            new_products = current_products + (product,)
            new_products_key = stringify_tuple(new_products)
            if new_products_key in result: # if already calculated for this set of products, skip it
                # print('stopped 2')
                continue

            # calculate
            occurrences = 0
            if new_products_key in basket_memo:
                occurrences = basket_memo[new_products_key]
                # print('memo', new_products, new_products_key, occurrences)
            else:
                for basket in baskets:
                    if set(new_products).issubset(basket):
                        occurrences += 1
                # print('save', new_products, new_products_key, occurrences)
                basket_memo[stringify_tuple(new_products)] = occurrences
            calculated_support = occurrences / len(baskets)
            # print(new_products, calculated_support)
            if calculated_support >= epsilon:
                result[new_products_key] = calculated_support
                recursive_all_support(new_products)
    
    # preprocess
    basket_tuples_keys = [stringify_tuple(basket) for basket in baskets]
    product_baskets_map = {}
    memo = {}
    
    for basket in baskets:
        for item in basket:
            if item in product_baskets_map:
                product_baskets_map[item].append(basket)
            else:
                product_baskets_map[item] = [basket]
    
    def iterative_support():
        for idx, cur_basket in enumerate(baskets):
            # if basket_tuples_keys[idx] in memo:
            #     continue
            # print(idx, '/', len(baskets))
            for possible_basket in powerset(cur_basket):
                current_tuple_key = stringify_tuple(possible_basket) # preprocess that
                if current_tuple_key in memo:
                    continue
                occurrences = 0
                set_possible_basket = set(possible_basket)
                for product in possible_basket:
                    for basket in product_baskets_map[product]:
                        if set_possible_basket.issubset(basket):
                            occurrences += 1
                calculated_support = occurrences / len(baskets)
                memo[current_tuple_key] = calculated_support
                if calculated_support < epsilon:
                    continue
                result[current_tuple_key] = calculated_support
            
            # current_tuple_key = basket_tuples_keys[idx]
            # 
            # occurrences = 0
            # for product in cur_basket:
            #     for basket in product_baskets_map[product]:
            #         if cur_basket.issubset(basket):
            #             occurrences += 1
            # calculated_support = occurrences / len(baskets)
            # memo[current_tuple_key] = calculated_support
            # if calculated_support < epsilon:
            #     continue
            # result[current_tuple_key] = calculated_support

    iterative_support()

    print('Total keys:', len(result.keys()))
    return result

supports = get_supports(baskets, EPSILON, 1)

Total keys: 63433


In [24]:
# definiujemy funkcje obliczajace support, confidence i lift

def support(supports, products: tuple[str]) -> float:
    # calculate support on demand
    products_key = stringify_tuple(products)
    if products_key in supports:
        # print('found support for', products_key, '->', supports[products_key])
        return supports[products_key]
    else:
        occurences = 0
        # print('calculating support for', products_key)
        products_set = set(products)
        for basket in baskets:
            if products_set.issubset(basket):
                occurences += 1
        calculated_support = occurences / len(baskets)
        supports[products_key] = calculated_support
        # print('just calculated:', occurences, len(baskets), calculated_support)
        return calculated_support
        

def confidence(supports, prior_products: tuple[str], following_products: tuple[str]) -> float:
    new_products = prior_products + following_products
    # print('confidence:', support(supports, new_products), support(supports, prior_products))
    return support(supports, new_products) / support(supports, prior_products)
    
def lift(supports, prior_products: tuple[str], following_products: tuple[str]) -> float:
    new_products = prior_products + following_products
    # print(support(supports, new_products) / (support(supports, prior_products) * support(supports, following_products)))
    return support(supports, new_products) / (support(supports, prior_products) * support(supports, following_products))

In [25]:
print(support(supports, ('whole milk', 'rolls/buns')))
print(confidence(supports, ('whole milk', 'rolls/buns'), ('yogurt',)))
print(lift(supports, ('whole milk', 'rolls/buns'), ('yogurt',)))

0.02793557441689501
0.14354066985645933
1.6714389440172768


## Część 3. - generowanie rekomendacji

In [26]:
# wyznaczamy liste potencjalnych rekomendacji
# rekomendowane artykuly powinny miec lift > 1 i mozliwie wysokie confidence

def generate_next_product_candidates(basket: set[str], products: list[str], supports: dict):
    result = []
    basket_tuple = tuple(basket)
    for product in products:
        if product in basket: #skip already used product
            continue
        if lift(supports, basket_tuple, (product,)) > 1:
            result.append((product, confidence(supports, basket_tuple, (product,)), lift(supports, basket_tuple, (product,))))
    # return result sorted by confidence
    return sorted(result, key=lambda x: x[1], reverse=True)
    

In [31]:
def pretty_print(chosen_basket, candidates):
    print(', '.join(chosen_basket))
    print(''.join(['-' for _ in range(82)]))
    for candidate in candidates:
        print('| {:>24} | {:^24} | {:<24} |'.format(*candidate))
    print()
# chosen_basket = {'sausage', 'frankfurter'}
chosen_basket = {'brown bread', 'butter'}
# chosen_basket = {'whole milk', 'coffee'}
pretty_print(chosen_basket, generate_next_product_candidates(chosen_basket, products, supports))

butter, brown bread
----------------------------------------------------------------------------------
|              canned beer |           0.3            | 6.394444444444445        |
|            baking powder |   0.19999999999999998    | 24.732231404958675       |
|             bottled beer |   0.19999999999999998    | 4.41386430678466         |
|                  dessert |   0.19999999999999998    | 8.477620396600567        |
|    fruit/vegetable juice |   0.19999999999999998    | 5.879371316306483        |
|                      ham |   0.19999999999999998    | 11.68984375              |
|            shopping bags |   0.19999999999999998    | 4.203089887640449        |
|                     soda |   0.19999999999999998    | 2.059600825877495        |
|            spread cheese |   0.19999999999999998    | 29.926                   |
|               whole milk |   0.19999999999999998    | 1.2664409648751587       |
|                     beef |   0.09999999999999999    | 2.945472440